In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time
import statsmodels.formula.api as sm
from statsmodels.formula.api import ols
import random

In [4]:
# Connect to an R session
import rpy2.robjects
r = rpy2.robjects.r
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
from rpy2.robjects.environments import Environment
from rpy2.robjects import pandas2ri
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
#Specify path with downloaded R packages
lib_path = 'C:/Users/BUCKBEAK/Documents/BUCKBEAK/R/win-library/3.2'

# load some required packages
utils = importr('utils')
langR = importr('languageR', lib_loc=lib_path)
lme4 = importr('lme4', lib_path)
lmerTest=importr('lmerTest', lib_path)

#Bring in all datasets from regressions

In [5]:
#7 days weighted
df = pd.read_csv('final_race_df_7daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_7daysweighted.csv')
weight_regdf7 = pd.merge(df, df2, how='left')
weight_regdf7 = weight_regdf7.drop(df.columns[0], axis=1)

#6 days weighted
df = pd.read_csv('final_race_df_6daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_6daysweighted.csv')
weight_regdf6 = pd.merge(df, df2, how='left')
weight_regdf6 = weight_regdf6.drop(df.columns[0], axis=1)

#5 days weighted
df = pd.read_csv('final_race_df_5daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_5daysweighted.csv')
weight_regdf5 = pd.merge(df, df2, how='left')
weight_regdf5 = weight_regdf5.drop(df.columns[0], axis=1)

#4 days weighted
df = pd.read_csv('final_race_df_4daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_4daysweighted.csv')
weight_regdf4 = pd.merge(df, df2, how='left')
weight_regdf4 = weight_regdf4.drop(df.columns[0], axis=1)

#3 days weighted
df = pd.read_csv('final_race_df_6daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_6daysweighted.csv')
weight_regdf3 = pd.merge(df, df2, how='left')
weight_regdf3 = weight_regdf3.drop(df.columns[0], axis=1)

#2 days weighted
df = pd.read_csv('final_race_df_6daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_6daysweighted.csv')
weight_regdf2 = pd.merge(df, df2, how='left')
weight_regdf2 = weight_regdf2.drop(df.columns[0], axis=1)

#DataFrames for the 5 individual days before the race
df = pd.read_csv('race_df_pre_weighting.csv')
dummies = pd.get_dummies(df['user_id'], prefix='user')
temp_df = pd.concat([df, dummies], axis=1)
race_times = pd.read_csv('huxc_race_times.csv')
race_times=race_times[race_times['user_id']!=2461]
race_times=race_times[race_times['user_id']!=2509]
reg_df = temp_df.merge(race_times, how='left')
reg_df = reg_df[reg_df['user_id']!=2439]
reg_df = reg_df.drop(['user_828'], axis=1)
reg_df = reg_df.drop(df.columns[0], axis=1).reset_index(drop=True)

reg_df0 = reg_df[reg_df['buildup_days']==0].reset_index(drop=True)
reg_df1 = reg_df[reg_df['buildup_days']==1].reset_index(drop=True)
reg_df2 = reg_df[reg_df['buildup_days']==2].reset_index(drop=True)
reg_df3 = reg_df[reg_df['buildup_days']==3].reset_index(drop=True)
reg_df4 = reg_df[reg_df['buildup_days']==4].reset_index(drop=True)
reg_df5 = reg_df[reg_df['buildup_days']==5].reset_index(drop=True)

#Exponentially Weighted DataFrames
rec_weight_df5 = pd.read_csv('expo_final_race_df_5daysweighted.csv')
wkout_weight_df5 = pd.read_csv('expo_workout_final_race_df_5daysweighted.csv')
expo_weight_df5 = pd.merge(rec_weight_df5, wkout_weight_df5, how='left')
expo_weight_df5 = expo_weight_df5.drop(df.columns[0], axis=1)

rec_weight_df4 = pd.read_csv('expo_final_race_df_4daysweighted.csv')
wkout_weight_df4 = pd.read_csv('expo_workout_final_race_df_4daysweighted.csv')
expo_weight_df4 = pd.merge(rec_weight_df4, wkout_weight_df4, how='left')
expo_weight_df4 = expo_weight_df4.drop(df.columns[0], axis=1)

rec_weight_df3 = pd.read_csv('expo_final_race_df_3daysweighted.csv')
wkout_weight_df3 = pd.read_csv('expo_workout_final_race_df_3daysweighted.csv')
expo_weight_df3 = pd.merge(rec_weight_df3, wkout_weight_df3, how='left')
expo_weight_df3 = expo_weight_df3.drop(df.columns[0], axis=1)

rec_weight_df2 = pd.read_csv('expo_final_race_df_2daysweighted.csv')
wkout_weight_df2 = pd.read_csv('expo_workout_final_race_df_2daysweighted.csv')
expo_weight_df2 = pd.merge(rec_weight_df2, wkout_weight_df2, how='left')
expo_weight_df2 = expo_weight_df2.drop(df.columns[0], axis=1)

Then grabbing variables that were significant, and put into one dataframe

###For Races
- REM sleep, runner mixed effects, two nights before the race: significant for both Bonferroni and Permutation Correction
- REM sleep, runner mixed effects, weighted two days: significant for Bonferroni only
- REM sleep, runner mixed effects, weighted three days: significant for Bonferroni only
- Total sleep, runner mixed effects, weighted three days: significant for Bonerroni only
- REM sleep, runner mixed effects, weighted four days: significant for Bonerroni only
- REM sleep, runner mixed effects, expo weighted 2 days: significant for both Bonferroni and Permutation Correction

###For Bike Efforts
- Slow wave sleep, runner mixed effects, 2nd night before the bike: significant for Bonferroni

In [6]:
REM_2nd_Night_Before = reg_df1[['user_id', 'race_period', 'rem_sleep_duration']]
REM_Weighted_Two_Days = weight_regdf2[['user_id', 'race_period', 'rem_sleep_duration']]
REM_Weighted_Three_Days = weight_regdf3[['user_id', 'race_period', 'rem_sleep_duration']]
REM_Weighted_Four_Days = weight_regdf4[['user_id', 'race_period', 'rem_sleep_duration']]
REM_Expo_Weighted_Two_Days = expo_weight_df2[['user_id', 'race_period', 'rem_sleep_duration']]
total_sleep_weighted_3days = weight_regdf3[['user_id', 'race_period', 'sleep_duration']]

#convert to R
pandas2ri.activate()
R_REM_2nd_Night_Before = pandas2ri.py2ri(REM_2nd_Night_Before)
R_REM_Weighted_Two_Days = pandas2ri.py2ri(REM_Weighted_Two_Days)
R_REM_Weighted_Three_Days = pandas2ri.py2ri(REM_Weighted_Three_Days)
R_REM_Weighted_Four_Days = pandas2ri.py2ri(REM_Weighted_Four_Days)
R_REM_Expo_Weighted_Two_Days = pandas2ri.py2ri(REM_Expo_Weighted_Two_Days)
R_total_sleep_weighted_3days = pandas2ri.py2ri(total_sleep_weighted_3days)

In [7]:
df = reg_df0
df = reg_df0.drop(['date_md', 'start', 'end', 'score', 'latency', 'time_in_bed', 'wake_duration', 'light_sleep_duration', 
                  'slow_wave_sleep_duration', 'rem_sleep_duration', 'cycles_count', 'debt_post', 'is_nap', 'recovery_score', 
                  'resting_heart_rate', 'hrv_rmssd', 'end_epoch', 'buildup_days', 'sleep_duration', 'disturbances'], axis=1)

#merge in variables of interest
df = df.merge(REM_2nd_Night_Before, how="left")
df.rename(columns = {'rem_sleep_duration':'REM_2nd_Night_Before'}, inplace = True)
df = df.merge(REM_Weighted_Two_Days, how="left")
df.rename(columns = {'rem_sleep_duration':'REM_Weighted_2_Days'}, inplace = True)
df = df.merge(REM_Weighted_Three_Days, how="left")
df.rename(columns = {'rem_sleep_duration':'REM_Weighted_3_Days'}, inplace = True)
df = df.merge(REM_Weighted_Four_Days, how="left")
df.rename(columns = {'rem_sleep_duration':'REM_Weighted_4_Days'}, inplace = True)
df = df.merge(REM_Expo_Weighted_Two_Days, how="left")
df.rename(columns = {'rem_sleep_duration':'REM_Expo_Weighted_2_Days'}, inplace = True)
df = df.merge(total_sleep_weighted_3days, how="left")
df.rename(columns = {'sleep_duration':'total_sleep_weighted_3days'}, inplace = True)

In [8]:
#add in average stress data

In [9]:
df.head()

,user_id,race_period,race_course,user_2439,user_2456,user_2458,user_2465,user_2466,user_2468,user_2469,user_2473,user_2508,seconds,pace_per_k,pace_time,FP_5K,Wisco_8K,Brown_8K,VCP_8K_Heps,FP_10K,VCP_8K_IC4A,REM_2nd_Night_Before,REM_Weighted_2_Days,REM_Weighted_3_Days,REM_Weighted_4_Days,REM_Expo_Weighted_2_Days,total_sleep_weighted_3days
0,828,1,FP_8K,0,0,0,0,0,0,0,0,0,1529.2,191.1500,03:11.1,0,0,0,0,0,0,5310000,4933928.57143,4933928.57143,5002000,5364000,29360357.1429
1,828,2,Brown_8K,0,0,0,0,0,0,0,0,0,1552.3,194.0375,03:14.0,0,0,1,0,0,0,1230000,3380769.23077,3380769.23077,2938000,744000,29561538.4615
2,828,3,VCP_8K_Heps,0,0,0,0,0,0,0,0,0,1582.5,197.8125,03:17.8,0,0,0,1,0,0,600000,1354615.38462,1354615.38462,1356000,492000,28440000.0000
3,828,4,FP_10K,0,0,0,0,0,0,0,0,0,1932.7,193.2700,03:13.3,0,0,0,0,1,0,2580000,4015714.28571,4015714.28571,4488000,8466000,34243928.5714
4,828,5,VCP_8K_IC4A,0,0,0,0,0,0,0,0,0,1617.6,202.2000,03:22.2,0,0,0,0,0,1,NaN,2973000.00000,2973000.00000,2420000,0,29991000.0000


#Mean-Centering the Signficant Regressions

###And switching units for better interpretations

In [34]:
#mean-centering pace per kilometer
mean_df = df.groupby('user_id').mean().reset_index()
mean_df = mean_df.drop(['race_period', 'user_2456', 'user_2458', 'user_2465', 'user_2466', 'user_2473', 'user_2508', 'user_2439', 
                        'user_2469', 'user_2468', 'FP_5K', 'Wisco_8K', 'Brown_8K', 'VCP_8K_Heps', 'FP_10K', 'VCP_8K_IC4A'], axis=1)
mean_df

,user_id,seconds,pace_per_k,REM_2nd_Night_Before,REM_Weighted_2_Days,REM_Weighted_3_Days,REM_Weighted_4_Days,REM_Expo_Weighted_2_Days,total_sleep_weighted_3days
0,828,1642.860000,195.694000,2430000,3331605.494506,3331605.494506,3240800.00000,3013200,30319364.83516
1,2456,1601.860000,190.694000,3525000,3481871.237748,3481871.237748,3633863.62303,2481600,26179369.54378
2,2458,1602.733333,200.341667,8544000,7528500.000002,7528500.000002,7899600.00000,8652000,28775785.71430
3,2465,1608.760000,191.505000,3846000,4066125.714286,4066125.714286,4204400.00000,3838800,27231932.34894
4,2466,1679.480000,199.923000,5466000,6581714.235724,6581714.235724,6574999.93334,6160800,25463428.50000
5,2468,1613.840000,192.164500,4305000,3776571.428570,3776571.428570,3825800.00000,2846400,29739357.08572
6,2469,1596.233333,199.529167,2700000,2445211.843712,2445211.843712,2470800.00000,2840400,29924211.80086
7,2473,1488.666667,186.083333,4920000,7661796.791446,7661796.791446,7834157.14286,7117200,24304914.05652
8,2508,1317.800000,200.365000,2298000,2138500.000000,2138500.000000,2151000.00000,1930800,28408976.19048


#Trying Multi-Variate Models

In [73]:
#Testing with weighted DF for 3 days
pandas2ri.activate()
r_df = pandas2ri.py2ri(df)
env=Environment()
xlist = ['REM_Weighted_3_Days', 'total_sleep_weighted_3days']
fit = stepwise_model(r_df, env, y_var='pace_per_k', x_vars=xlist, group_vars='user_id')
print r.summary(fit)

Linear mixed model fit by REML t-tests use Satterthwaite approximations to

  degrees of freedom [lmerMod]

Formula: pace_per_k ~ +REM_Weighted_3_Days + total_sleep_weighted_3days +  

    (1 | user_id)



REML criterion at convergence: 284.2



Scaled residuals: 

     Min       1Q   Median       3Q      Max 

-1.78067 -0.52690 -0.09394  0.65317  2.01614 



Random effects:

 Groups   Name        Variance Std.Dev.

 user_id  (Intercept) 35.37    5.947   

 Residual             24.90    4.990   

Number of obs: 36, groups:  user_id, 9



Fixed effects:

                             Estimate Std. Error         df t value Pr(>|t|)

(Intercept)                 2.239e+02  1.030e+01  3.198e+01  21.736   <2e-16

REM_Weighted_3_Days        -8.579e-07  3.273e-07  3.272e+01  -2.621   0.0132

total_sleep_weighted_3days -8.819e-07  3.723e-07  3.149e+01  -2.369   0.0241

                              

(Intercept)                ***

REM_Weighted_3_Days        *  

total_sleep_weighted_3days *  



#The Functions We Need From Previous Notebooks

In [10]:
def mixed_effects_model(dftouse, env, y_var, x_var, group_vars=None):
    env = env
    if group_vars == None:
        print 'Not a mixed effects model!'
        return 'Not a mixed effects model!'
    for varname in r.colnames(dftouse):
        env[varname] = dftouse.rx2(varname)
    if type(group_vars) == str:
        formula = Formula(y_var + ' ~ ' + x_var + ' + (1|' + group_vars + ')', environment = env)
        model = lmerTest.lmer(formula)
        return r.summary(model)
    elif type(group_vars)== tuple or type(group_vars)== list:
        if len(group_vars) == 2:
            formula = Formula(y_var + ' ~ ' + x_var + ' + (1|' + group_vars[0] + ') + (1|' + group_vars[1] + ')', environment=env)
            model = lmerTest.lmer(formula)
            return r.summary(model)
        elif len(group_vars) == 3:
            formula =Formula(y_var + ' ~ ' + x_var + ' + (1|' + group_vars[0] + ' ' + group_vars[1] + ' ' + group_vars[2]+ ')', 
                             environment = env)
            model = lmerTest.lmer(formula)
            return r.summary(model)
    

#Trying a Stepwise Model with all variables

However, this was leading to errors

In [43]:
env=Environment()
xlist = ['hrv_rmssd', 'resting_heart_rate', 'light_sleep_duration', 'rem_sleep_duration', 'slow_wave_sleep_duration',
'cycles_count', 'latency', 'z1', 'z2', 'z3', 'z4', 'z5']
for varname in r.colnames(reg_test):
    env[varname] = reg_test.rx2(varname)
x_string= ' + '
for i in range(0, len(xlist)):
    if i != (len(xlist)-1):
        x_string+= xlist[i] + ' + '
    else:
        x_string+= xlist[i] + ' '
formula = Formula('pace_per_k' + ' ~ ' + x_string + ' + (1|user_id)', environment = env)
fit = lmerTest.lmer(formula)
stepwise_model = r.step(fit, direction='both')

RRuntimeError: Error in solve.default(xtx[inds, inds]) : 
  system is computationally singular: reciprocal condition number = 5.51812e-19


In [20]:
xlist = ['hrv_rmssd', 'resting_heart_rate', 'light_sleep_duration', 'rem_sleep_duration', 'sleep_duration', 'slow_wave_sleep_duration',
'cycles_count', 'time_in_bed', 'latency', 'z1', 'z2', 'z3', 'z4', 'z5']

In [63]:
def stepwise_model(dftouse, env, y_var, x_vars, group_vars=None):
    if group_vars==None:
        print 'Not a mixed effects model!'
        return 'Not a mixed effects model!'
    env=env
    for varname in r.colnames(dftouse):
        env[varname] = dftouse.rx2(varname)
    
    x_string= ' + '
    for i in range(0, len(xlist)):
        if i != (len(xlist)-1):
            x_string+= xlist[i] + ' + '
        else:
            x_string+= xlist[i] + ' '
    
    if type(group_vars) == str:
        formula = Formula(y_var + ' ~ ' + x_string + ' + (1|' + group_vars + ')', environment = env)
        fit = lmerTest.lmer(formula)
        return fit
    
    elif type(group_vars)== tuple or type(group_vars)== list:
        if len(group_vars) == 2:
            formula = Formula(y_var + ' ~ ' + x_string + ' + (1|' + group_vars[0] + ') + (1|' + group_vars[1] + ')', environment=env)
            fit = lmerTest.lmer(formula)
            return fit
        
        elif len(group_vars) == 3:
            formula =Formula(y_var + ' ~ ' + x_string + ' + (1|' + group_vars[0] + ' ' + group_vars[1] + ' ' + group_vars[2]+ ')', 
                             environment = env)
            fit = lmerTest.lmer(formula)
            return fit

In [26]:
env=Environment()
fit = stepwise_model(reg_test, env, 'pace_per_k', xlist, group_vars='user_id')

RRuntimeError: Error: Dropping columns failed to produce full column rank design matrix
